### Generative adversarial network

In this notebook, I train a GAN on the MNIST dataset of handwritten digits.

Much of the code is copied from similar examples online.

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

## Prepare MNIST data

# Use both the training & test MNIST digits.
batch_size = 16
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
all_digits = all_digits[0:2000]
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(32)


In [17]:
## Create the discriminator
#It maps 28x28 digits to a binary classification score (i.e. a score for how realistic it is)

discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Print structure of discriminator:
discriminator.summary()

## Create the generator
#It mirrors the discriminator, replacing `Conv2D` layers with `Conv2DTranspose` layers.
# These are deconvolutional layers, which do the reverse of convolutional ones.

latent_dim = 128

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

# Print structure of generator
generator.summary()


Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 74,625
Trainable params: 74,625
Non-trainable params: 0
_________________________________________________

In [18]:

## Override `train_step`

class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        if isinstance(real_images, tuple):
            real_images = real_images[0]
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        print("Batch size: {}".format(batch_size))
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images" (which is what we want the generator to output)
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (we want it to create images the discriminator thinks are real)
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))
        return {"d_loss": d_loss, "g_loss": g_loss}



In [19]:
## Create a callback that periodically saves generated images

class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(generated_images[i])
            img.save("generated/generated_img_{i}_{epoch}.png".format(i=i, epoch=epoch))


## Train the end-to-end model

epochs = 20

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=3, latent_dim=latent_dim)]
)


# Display the last generated images:

from IPython.display import Image, display

display(Image("generated/generated_img_0_{}.png".format(epochs-1)))
display(Image("generated/generated_img_1_{}.png".format(epochs-1)))
display(Image("generated/generated_img_2_{}.png".format(epochs-1)))

Epoch 1/20
Batch size: Tensor("strided_slice:0", shape=(), dtype=int32)
Batch size: Tensor("strided_slice:0", shape=(), dtype=int32)
125/125 [==============================] - 241s 2s/step - d_loss: 0.3501 - g_loss: 1.0751
Epoch 2/20
125/125 [==============================] - 132s 1s/step - d_loss: 0.0315 - g_loss: 4.2238
Epoch 3/20
125/125 [==============================] - 183s 1s/step - d_loss: 0.4578 - g_loss: 2.9995
Epoch 4/20
125/125 [==============================] - 191s 2s/step - d_loss: 0.4878 - g_loss: 1.2618
Epoch 5/20
125/125 [==============================] - 123s 987ms/step - d_loss: 0.4800 - g_loss: 1.3490
Epoch 6/20
125/125 [==============================] - 252s 2s/step - d_loss: 0.6055 - g_loss: 1.0360
Epoch 7/20
125/125 [==============================] - 113s 902ms/step - d_loss: 0.6644 - g_loss: 1.0027
Epoch 8/20
125/125 [==============================] - 207s 2s/step - d_loss: 0.5735 - g_loss: 1.1738
Epoch 9/20
125/125 [==============================] - 172s 1s/st